In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np 
import pandas as pd 
import scanpy as sc

import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
import sys, os
sys.path.append('../src')

# from interaction import Interaction
from util import compute_auc

In [164]:
# # JING CLONAL EXPANSION
# x_path = '/ix/djishnu/Jane/SLIDESWING/jing_data/KIR+TEDDY/data/KIR+TEDDY_rna_filtered85.csv'
# y_path = '/ix/djishnu/Jane/SLIDESWING/jing_data/KIR+TEDDY/data/KIR+TEDDY_Yexpanded_filtered85.csv'
# slide_outs = '/ix/djishnu/Jane/SLIDESWING/jing_data/KIR+TEDDY/KIR+TEDDY_filtered85/KIR+TEDDY_filtered85_noint_output/0.01_0.5_out'

# # JING TUMOR TIL VS TEMRA
# x_path = '/ix/djishnu/alw399/SLIDE_PLM/data/jing_tumor/tumor_x2.csv'
# y_path = '/ix/djishnu/alw399/SLIDE_PLM/data/jing_tumor/tumor_y2.csv'
# slide_outs = '/ix/djishnu/alw399/SLIDE_PLM/data/jing_tumor/0.05_0.5_out'

# ALOK ANTIGEN SPECIFICITY 
x_path = '/ix/djishnu/Jane/SLIDESWING/alok_data/data/Ins1_InsChg2_rna_MRfilt_forSLIDE.csv'
y_path = '/ix/djishnu/Jane/SLIDESWING/alok_data/data/Ins1_InsChg2_rna_MRfilt_antigens.csv'
slide_outs = '/ix/djishnu/Jane/SLIDESWING/alok_data/alok_data12_MRfilt_noint_out/0.01_2_out'


In [165]:
# y = pd.read_csv(y_path)['Y'].values
# y = pd.read_csv(y_path)['y'].values
y = pd.read_csv(y_path)['Antigen'].values - 1
y.shape

(73,)

In [166]:
from util import get_genes_from_slide_outs

lf_dict = get_genes_from_slide_outs(slide_outs)
lf_dict.keys()

dict_keys(['Z30', 'Z104', 'Z109', 'Z149', 'Z206'])

In [167]:
all_genes = np.unique(np.concatenate([lf_dict[lf] for lf in lf_dict]))
len(all_genes)

60

In [168]:
from genept import GenePTEmbedder

genept = GenePTEmbedder()
gene_embeddings = genept.get_gene_info(all_genes)
gene_embeddings.shape

(60, 1536)

In [169]:
# Create 0, 1 presence/absence matrix

gex_df = pd.read_csv(x_path, usecols=list(all_genes))
gex_threshes = gex_df.mean(axis=0)

mask_df = pd.DataFrame(
    np.where(gex_df > gex_threshes, 1, 0), 
    index=gex_df.index, 
    columns=gex_df.columns
)

mask_df.shape

(73, 60)

In [170]:
genept_df = np.einsum('ij,jk->ijk', mask_df.values, gene_embeddings)

genept_df = genept_df.reshape(gex_df.shape[0], -1)
genept_df.shape

(73, 92160)

In [171]:
wgenept_df = gex_df @ gene_embeddings
wgenept_df.shape

(73, 1536)

In [172]:
z_matrix = pd.read_csv(os.path.join(slide_outs, 'z_matrix.csv'), index_col=0)
z_matrix = z_matrix[list(lf_dict.keys())].values
z_matrix.shape

(73, 5)

In [186]:
# from models import Estimator
from sklearn.linear_model import Lasso, LinearRegression

# a=0.1
# model = Lasso(alpha=a)
model = LinearRegression()

In [187]:
# SLIDE z-matrix performance
lasso0 = Estimator(model=model)

scores = lasso0.evaluate(z_matrix, y)
auc0 = np.mean(scores)
auc0

0.6466880341880341

In [188]:
# Lasso regression on LF gene expression matrix
lasso1 = Estimator(model=model)

scores = lasso0.evaluate(gex_df.values, y)
auc1 = np.mean(scores)
auc1

0.601466588966589

In [189]:
# Lasso regression on mean-thresholded gene expression

lasso2 = Estimator(model=model)

scores = lasso2.evaluate(mask_df.values, y)
auc2 = np.mean(scores)
auc2

0.5185703185703187

In [190]:
# Lasso regression on semantic embeddings

lasso3 = Estimator(model=model)

scores = lasso3.evaluate(wgenept_df.values, y)
auc3 = np.mean(scores)
auc3

0.5

In [191]:
# with open(os.path.join(slide_outs, 'standard_out.txt'), 'r') as f:
#     standard_out = f.readlines()
# slide_auc = standard_out[-1].split(' ')[-2]

# slide_auc = '0.951218206396577'
slide_auc = '0.747932'

In [192]:
pd.DataFrame({
    'slide': slide_auc[:8],
    'z_matrix': auc0,
    'gex': auc1,
    'mask_gex': auc2,
    'wgenept': auc3
}, index=['auc']).T


,auc
slide,0.747932
z_matrix,0.646688
gex,0.601467
mask_gex,0.51857
wgenept,0.5
